In [ ]:
import random
import numpy as np
from tqdm import tqdm
from keras.datasets import mnist

In [ ]:
def standardize(data, ax):
    mean = np.sum(data, axis=ax) / len(data)
    variance = np.sum((data - mean) ** 2) / len(data)
    std_deviation = np.sqrt(variance)

    standardized_data = (data - mean) / std_deviation
    return standardized_data

In [ ]:
def K_fold(data, K=5):
    fold_size = data.shape[0] // K
    folds = np.zeros([K, fold_size, 785])
    ready_data = np.array(data, dtype=object)
    for i in range(K):
        indices = np.random.choice(ready_data.shape[0], size=fold_size, replace=False)
        work_fold = ready_data[indices]
        folds[i] = np.array(work_fold)
        ready_data = np.delete(ready_data, indices, axis=0)
    folds = np.array(folds)
    return folds

In [ ]:
def rand_weights(size):
    dum = list()
    for i in range(size):
        dum.append(random.uniform(-1, 1))
    return np.array(dum)


def sigmoid(x):
    X = [(1 / (1 + np.exp(-z))) for z in x]
    return np.array(X)


def error(H, Y):
    er = 0
    for i in range(len(H)):
        er = er + ((Y[i] * np.log(H[i])) + ((1 - Y[i]) * np.log(1 - H[i])))
    return round(er / len((H)), 4)


def accuracy(class_f, Y_test):
    return (np.sum(class_f == Y_test) / len(Y_test)) * 100


class LogisticRegression():
    def __init__(self, learning_rate=0.05, maxIter=1000, error_ratio=0.01):
        self.__learning_rate = learning_rate
        self.__maxIter = maxIter
        self.__weigths = None
        self.__bias = 0
        self.__error_ratio = error_ratio

    def fit(self, X, Y):
        sample_size = np.array(X).shape[0]
        n_features = np.array(X).shape[1]
        #self.__weigths = np.zeros(n_features)
        self.__weigths = rand_weights(n_features)
        Error = 1
        epoch_bar = tqdm(desc='Epochs', total=self.__maxIter)
        for i in range(self.__maxIter):
            epoch_bar.update(1)
            epoch_bar.set_postfix({'accuracy': f'{1 - Error:.3f}'})
            linear = np.dot(X, self.__weigths) + self.__bias
            prediction = sigmoid(linear)
            dw = (1 / sample_size) * np.dot(X.T, (prediction - Y))
            db = (1 / sample_size) * np.sum(prediction - Y)

            self.__weigths = self.__weigths - self.__learning_rate * dw
            self.__bias = self.__bias - self.__learning_rate * db
            Error = abs(error(prediction, Y))
            if self.__error_ratio > Error:
                print(Error)
                break

    def predict(self, X_test):
        linear = np.dot(X_test, self.__weigths) + self.__bias
        Y_predicted = sigmoid(linear)
        class_f = [1 if y > 0.5 else 0 for y in Y_predicted]
        return class_f

    def get_weights(self):
        return self.__weigths

    def set_weights(self, weights):
        self.__weigths = weights

    def get_learning_rate(self):
        return self.__learning_rate

    def set_learning_rate(self, learning_rate):
        self.__learning_rate = learning_rate

In [ ]:
def model(x_train, y_train, x_test, y_test, l_r):
    model = LogisticRegression(learning_rate=l_r, maxIter=1000)
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    acc = accuracy(pred, y_test)
    return acc, model

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
train_X = np.append(train_X[np.where(train_y == 0)], train_X[np.where(train_y == 1)], axis=0)
train_y = np.append(train_y[np.where(train_y == 0)], train_y[np.where(train_y == 1)], axis=0).reshape(-1, 1)
test_X = np.append(test_X[np.where(test_y == 0)], test_X[np.where(test_y == 1)], axis=0)
test_y = np.append(test_y[np.where(test_y == 0)], test_y[np.where(test_y == 1)], axis=0).reshape(-1, 1)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(12665, 28, 28) (12665, 1) (2115, 28, 28) (2115, 1)


In [ ]:
x_train = train_X.reshape(-1, 28 * 28)
x_train = np.array(standardize(x_train, 0))
x_test = test_X.reshape(-1, 28 * 28)
x_test = np.array(standardize(x_test, 0))
print(x_train.shape, x_test.shape)

(12665, 784) (2115, 784)


In [ ]:
train_data = np.append(x_train, train_y, axis=1)
print(train_data.shape)

(12665, 785)


In [ ]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]
validation_accuarcy = []
test_accuarcy = []
models = []

In [ ]:
for i in learning_rates:
    training = K_fold(train_data, 10)
    for j in range(10):
        val = training[j]
        x_val = np.array(val[:, 0:784])
        y_val = np.array(val[:, 784])

        train = np.delete(training, j, axis=0).reshape(-1, 785)
        x_train = np.array(train[:, 0:784])
        y_train = np.array(train[:, 784])

        Acc, Model = model(x_train, y_train, x_val, y_val, i)

        validation_accuarcy.append(Acc)
        models.append(Model.get_weights())

Epochs: 100%|██████████| 1000/1000 [02:40<00:00,  6.25it/s, accuracy=0.439]


In [ ]:
Max = max(validation_accuarcy)
idx = validation_accuarcy.index(Max)
weights = models[idx]
learning_rate = learning_rates[(idx // 10)]
print(Max, idx, learning_rate)

99.76303317535546 2 0.1


In [ ]:
cls = LogisticRegression(learning_rate=learning_rate)
cls.set_weights(weights)
predictions = cls.predict(x_test)
print(f"acc on test = {accuracy(predictions, test_y)}")

acc on test = 106376.59574468085
